In [12]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [13]:
import os
import json

In [14]:
import semantic_kernel as sk
from dotenv import load_dotenv
import semantic_kernel.connectors.ai.open_ai as skaoai

In [15]:
kernel = sk.Kernel()
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [16]:
kernel.add_chat_service("azure_chat_competion_service", skaoai.AzureChatCompletion("Your Azure OpenAI Service gpt-4-32k Model Delployment Name",endpoint,api_key=api_key,api_version = "2023-12-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", skaoai.AzureTextEmbedding("Your Azure OpenAI Service text-embedding-ada-002 Model Delployment Name", endpoint,api_key=api_key,api_version = "2023-07-01-preview")
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'azure_chat_competion_service': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x1305b60e0>}, chat_services={'azure_chat_competion_service': <function Kernel.add_chat_service.<locals>.<lambda> at 0x1305b6050>}, text_embedding_generation_services={'embeddings_services': <function Kernel.add_text_embedding_generation_service.<locals>.<lambda> at 0x1305b4ee0>}, default_text_completion_service='azure_chat_competion_service', default_chat_service='azure_chat_competion_service', default_text_embedding_generation_service='embeddings_services', retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [17]:
base_plugin = "./plugins"

In [18]:
files_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "FilePlugin")

In [19]:
nodes_files = os.listdir("./data/notes")
transcrips_files = os.listdir("./data/transcripts")

In [20]:
kblist = []

In [21]:
for f in nodes_files:
    file = open("./data/notes/"+f, "r") 
    content = file.read()
    notesFunc = files_plugin["Notes"]
    result = await notesFunc(content)
    print(result.result.replace("\n", " ")) 
    json_result = json.loads(result.result)
    kblist.append(json_result)
    file.close()

{"kb": "History of machine learning", "content": "The history of machine learning and artificial intelligence is marked by significant milestones. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). Alan Turing laid the foundation for the concept of a 'machine that can think' in 1950. The Dartmouth Summer Research Project in 1956 was a seminal event for AI as a field. The period from 1956 to 1974 is often referred to as 'The golden years' of AI. However, by the mid-1970s, the complexity of creating 'intelligent machines' had been understated and its promise overblown, leading to an 'AI Winter'. The 1980s saw the proliferation of 'expert systems'. The period from 1993 to 2011 saw a new era for ML and AI to solve some of the problems caused earlier by the lack of data and compute power. Today, machine learning and AI touch almost 

In [22]:
for f in transcrips_files:
    file = open("./data/transcripts/"+f, "r") 
    content = file.read()
    transcripsFunc = files_plugin["Transcrips"]
    result = await transcripsFunc(content)
    print(result.result)
    json_result = json.loads(result.result)
    for item in json_result:
        kblist.append(item)
    file.close()

[
{"kb": "Alan Turing and the Turing test", "content": "Alan Turing, Turing test, machine intelligence, interrogator, human and computer"},
{"kb": "The Dartmouth Summer Research Project on AI", "content": "Artificial intelligence, Dartmouth College, summer research, AI field, 1956 to 1974"},
{"kb": "The golden years of AI", "content": "AI optimism, Marvin Minsky, natural language processing, micro worlds, intelligent machines"},
{"kb": "The AI winter", "content": "Complexity of AI, limited compute power, lack of data, ethics of AI, AI winter"},
{"kb": "Resurgence and fall of AI for expert systems", "content": "Expert systems, resurgence in AI, rule-based inference systems, machine intelligence, personal computers"},
{"kb": "Growth in AI driven by more data and more powerful hardware", "content": "Compute and storage capabilities, internet, smartphones, machine learning, computer vision"},
{"kb": "Big Data", "content": "Compute power, size of data sets, machine learning, solving problem

In [23]:
kblist

[{'kb': 'History of machine learning',
  'content': "The history of machine learning and artificial intelligence is marked by significant milestones. Notable discoveries include Bayes Theorem (1763, 1812), Least Square Theory (1805), Markov Chains (1913), Perceptron (1957), Nearest Neighbor (1967), Backpropagation (1970), and Recurrent Neural Networks (1982). Alan Turing laid the foundation for the concept of a 'machine that can think' in 1950. The Dartmouth Summer Research Project in 1956 was a seminal event for AI as a field. The period from 1956 to 1974 is often referred to as 'The golden years' of AI. However, by the mid-1970s, the complexity of creating 'intelligent machines' had been understated and its promise overblown, leading to an 'AI Winter'. The 1980s saw the proliferation of 'expert systems'. The period from 1993 to 2011 saw a new era for ML and AI to solve some of the problems caused earlier by the lack of data and compute power. Today, machine learning and AI touch almo

In [24]:
base_vectordb = 'aboutMLKBDemoDemo'

In [25]:

from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [26]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection(base_vectordb)

In [27]:
kernel.register_memory_store(memory_store=qdrant_store)

In [28]:
import uuid

In [29]:
uuid.uuid4()

UUID('0c05f6de-e1bf-4d32-be38-d7fdd9706a17')

In [30]:
for item in kblist:
    content = item["kb"] + ' - ' + item["content"]
    id =str(uuid.uuid4())
    await kernel.memory.save_information(base_vectordb, id=id, text=content)

In [31]:
ask = "can you tell me what is different ML and AI"

memories = await kernel.memory.search(
    base_vectordb, ask, limit=1, min_relevance_score=0.8
)

In [32]:
result = ''
for memory in memories:
    print(f"Top Result: {memory.text} with score {memory.relevance}")
    result = memory.text

Top Result: The difference between AI and ML - AI accomplishes human tasks, ML subset of AI, ML uses algorithms, deep learning subset of ML, neural networks learning with score 0.86017144


In [33]:
answer_plugin = kernel.import_semantic_plugin_from_directory(base_plugin , "AnswerPlugin")

In [34]:
answerFunc = answer_plugin["Summary"]

In [35]:
summary_result = await answerFunc(result)

In [36]:
summary_result.result

"AI, or Artificial Intelligence, is a broad concept that involves machines performing tasks in a way that mimics human intelligence. Machine Learning (ML), a subset of AI, involves the use of algorithms to allow machines to learn from data. Deep Learning, a further subset of ML, involves the use of neural networks to mimic the human brain's learning process. In essence, all these technologies work together to enable machines to perform tasks that would normally require human intelligence."